In [1]:
# -*- coding:utf-8 -*-
"""
Description: 
    1) Use the pre handled data(image description and image features to train the caption geneartion model
    2) firstly quick choose a model to train
Author: allocator
"""

'\nDescription: \n    1) Use the pre handled data(image description and image features to train the caption geneartion model\n    2) firstly quick choose a model to train\nAuthor: allocator\n'

In [2]:
import keras
import numpy as np
import os
import numpy.random as rd
import json
import h5py

Using TensorFlow backend.


In [3]:
img_dir = '../data/img'
img_feature_dir = '../data/img_feature'
img_feature_file = 'img_features.h5'
clean_txt_dir = '../data/clean_txt'
clean_txt_file = 'image_descs.json'
set_category = 200
seed = 10
output_dir = '../data/res'
output_filename = 'development_dataset_id.json'

In [4]:
# first seperate the data set
def seperate_dataset(img_dir, category, seed):
    """Generate the development set to quickly choose the model and configuration"""
    img_list = os.listdir(img_dir)
    # random select the train set and test set from the image list
    img_list = [item.split('.')[0] for item in img_list]
    img_size = len(img_list)
    extract_set = set()
    dataset = {}
    rd.seed(seed)
    while len(extract_set) < category:
        curr_id = rd.randint(img_size)
        if curr_id not in extract_set:
            extract_set.add(curr_id)
    print(' extract set generated and length %d ' % len(extract_set))
    set_len = int(category/2)
    dataset['train'] = list(extract_set)[:set_len]
    dataset['test'] = list(extract_set)[set_len:]
    return dataset

In [5]:
# save the dataset id
def save_dataset(filename, dataset):
    """Save the dataset."""
    file_path = os.path.join(output_dir, filename)
    json.dump(dataset, open(file_path, 'w'))

In [ ]:
def load_txt(filename, dataset):
    train_set = {}
    test_set = {}
    train_id_list = dataset['train']
    test_id_list = dataset['test']
    file_path = os.path.join(clean_txt_dir, filename)
    image_descs = json.load(file_path)
    for item in train_id_list:
        train_set[item] = 'startseq ' + ' '.join(image_descs[item]) + ' endseq'
    for item in test_id_list:
        test_set[item] = 'startseq ' + ' '.join(image_descs[item]) + ' endseq'
    return train_set, test_set

In [ ]:
# load the features about the images
def load_image_feature(filename, dataset):
    """Load the image feature about the train and test image id list."""
    image_feature_path = os.path.join(filename)
    image_features = h5py.File(image_feature_path, 'r')
    train_set = {}
    test_set = {}
    train_id_list = dataset['train']
    test_id_list = dataset['test']
    for item in train_id_list:
        train_set[item] = np.array(image_features[item])
    for item in test_id_list:
        test_set[item] = np.array(image_features[item])
    return train_set, test_set

In [6]:
dataset = seperate_dataset(img_dir, set_category, seed)
# save_dataset(output_filename, dataset)

 extract set generated and length 200 


TypeError: 'set' object is not subscriptable

In [ ]:
train_desc, test_desc = load_txt(os.path.join(clean_txt_dir, clean_txt_file), dataset)
train_img, test_img = load_image_feature(os.path.join(img_feature_dir, img_feature_file), dataset)

In [ ]:
# some information about the trainging and test data set
print(' train_desc length %d test_desc length %d ' % (len(train_desc), len(test_desc)))
print(' train_img length %d test_img length %d ' % (len(train_img), len(test_img)))